In [1]:
import tarfile

dump_path = '/mnt/large/small_genomes/taxdump.tar.gz'

dump = tarfile.open(dump_path, "r:gz")

In [2]:
names = dump.extractfile(dump.getmember('names.dmp')).read().split(b'\t|\n')
len(names)

1984035

In [3]:
tax_id_names = {}

for l in names:
    if len(l) == 0:
        continue
        
    tokens = l.split(b'\t|\t')
    
    taxid = int(tokens[0])
    name = str(tokens[1])
    unique = str(tokens[2])
    name_class = str(tokens[3])
    
    n = tax_id_names.setdefault(taxid, {})
    
    if name_class != 'misspelling' and name_class != 'type material':
        n[name_class] = name

In [44]:
from collections import Counter

findings_path = '/mnt/large/small_genomes/informatives_lca1_genus.txt'

with open(findings_path) as f:
    contents = f.read()

In [45]:
taxid_file_counts = []

for l in contents.split('\n'):
    if len(l) == 0:
        continue;
        
    tokens = l.split(':')
    
    read_header = tokens[0]
    tax_id = int(tokens[1])
    
    counts = [int(c) for c in read_header.split('_')[1:]]
    
    if len(taxid_file_counts) == 0:
        for _ in range(0, len(counts)):
            taxid_file_counts.append(list())
            
    for i, c in enumerate(counts):
        for _ in range(0, c):
            taxid_file_counts[i].append(tax_id)
        
taxid_file_counts = [Counter(c) for c in taxid_file_counts]

In [46]:
all_tax_ids = set()

for c in taxid_file_counts:
    for taxid in c:
        all_tax_ids.add(taxid)

In [47]:
data = [[taxid, tax_id_names.get(taxid, {}).get("b'scientific name'", "b''")[2:-1]] for taxid in all_tax_ids]

In [48]:
for l in data:
    for c in taxid_file_counts:
        l.append(c.get(l[0], 0))

In [49]:
for l in data:
    total = sum(l[2:])
    l.insert(0, total)

In [50]:
in_order = sorted(data, reverse=True)

In [51]:
from os.path import basename, splitext

input_files = [
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR1270040.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR1284500.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2084430.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2155541.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2164311.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2167785.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2171483.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2174564.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2175366.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2180979.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2182179.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR2182227.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR772892.fastq.gz',
    '/scratch/vyf2/CR2/Reads/ready/Rmts_bact_SRR772893.fastq.gz',
]

input_files = [basename(p).split('.')[0] for p in input_files]

len(input_files) == len(in_order[0][3:])

True

In [52]:
with open('/mnt/large/small_genomes/counts_lca1_genus.csv', 'w') as f:
    headers = ['taxid', 'scientific name', 'total count']
    for fn in input_files:
        headers.append(fn)
        
    f.write(','.join(headers) + '\n')
    for record in in_order:
        f.write(str(record[1]))
        f.write(',')
        f.write(str(record[2]))
        f.write(',')
        f.write(str(record[0]))
        
        for count in record[3:]:
            f.write(',')
            f.write(str(count))
            
        f.write('\n')